In [ ]:
import matplotlib
matplotlib.use('Qt5Agg')
from matplotlib import pyplot as plt
plt.rcParams.update({'lines.linewidth':2.5})
plt.rcParams.update({'font.size': 18})
plt.rcParams['figure.figsize'] = (7,7)

import numpy as np
import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')
from parse_density_files import CUBE


# dsgdb9nsd_002626

In [ ]:
ve38 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_38.cube')
ve30 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_30.cube')
ve23 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_23.cube')
ve15 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_15.cube')
ve8 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_8.cube')
ve0 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/free_electron_gas.cube')

In [ ]:
import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')
from alchemy_tools import atomic_energy_decomposition
p38 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_38.cube'
p30 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_30.cube'
p23 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_23.cube'
p15 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_15.cube'
p8 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_8.cube'
p0 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/free_electron_gas.cube'
cubes = [(p0, 0), (p8, 8/38), (p15, 15/38), (p23, 23/38), (p30, 30/38), (p38, 1)]
# ae, alchpots = atomic_energy_decomposition(cubes)
atomic_energy_decomposition(cubes, '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/output')

In [ ]:
print(ae)

# Integration

In [ ]:
from alchemy_tools import integrate_lambda_density, calculate_atomic_energies, calculate_alchemical_potential

In [ ]:
lambda_densities = [ve0, ve8, ve15, ve23, ve30, ve38]
density_arrays = []
for dens in lambda_densities:
    dens.scale()
    density_arrays.append(dens.data_scaled)
lam_vals = [0, 8/38, 15/38, 23/38, 30/38, 1]

In [ ]:
av_dens = integrate_lambda_density(density_arrays, lam_vals)

# Plots

In [ ]:
# projection on line
x = np.linspace(0, 20, 175)
sa = (1,2)
av_dens_1D = av_dens.sum(axis=sa)
pd_1D = [ve38.project(sa), ve30.project(sa), ve23.project(sa), ve15.project(sa), ve8.project(sa), ve0.project(sa), av_dens_1D]
label = [r'$\rho(\lambda = 1.0)$',r'$\rho(\lambda \approx 0.79)$', r'$\rho(\lambda \approx 0.61)$', r'$\rho(\lambda \approx 0.39)$', r'$\rho(\lambda \approx 0.21)$', r'$\rho(\lambda = 0)$', r'$\tilde{\rho} \approx \int_0^1 d\lambda \rho(\lambda, r)$']
fig, ax = plt.subplots(1,1)
for idx, pd in enumerate(pd_1D):
    # bigger linewidth for lambda-averaged density
    if idx == len(pd_1D)-1:
        ax.plot(x, pd, '--', label=label[idx])
    else:
        ax.plot(x, pd, label=label[idx])

# ax.set_title('Boxsize = 20 Ang')
ax.set_xlabel('Cell coordinate (Ang)')
ax.set_xlim(0.5, 19.5)
ax.set_ylabel(r'Projected Density $\rho(x_0)$ (Bohr$^{-3}$) ')

ax.legend()

# Alchemical Potential and Atomic Energies

In [ ]:
nuclei = ve38.atoms
meshgrid = ve38.get_grid()
meshgrid_xyz = np.vstack([_.flatten() for _ in meshgrid]).T

In [ ]:
nuc=np.array(ve38.atoms)[0][1:].T

In [ ]:
np.array(ve38.atoms)[0][1:4]

In [ ]:
calculate_alchemical_potential(av_dens, meshgrid, nuc)

In [ ]:
# %timeit atomic_energies = get_atomic_energies(av_dens, nuclei, meshgrid)
atomic_energies = calculate_atomic_energies(av_dens, nuclei, meshgrid)

In [ ]:
# %timeit dist_gpt_nuc = np.linalg.norm(meshgrid_xyz - nuclei[0][1:], axis=1)
#(density.flatten()/dist_gpt_nuc).sum()


In [ ]:
atomic_energies

# Atomisation Energies

In [ ]:
import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')
from alchemy_tools import atomic_energy_decomposition
p38 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_38.cube'
p30 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_30.cube'
p23 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_23.cube'
p15 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_15.cube'
p8 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_8.cube'
p0 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/free_electron_gas.cube'
cubes = [(p0, 0), (p8, 8/38), (p15, 15/38), (p23, 23/38), (p30, 30/38), (p38, 1)]
# cubes = [(p0, 0)]
# atomic energies and alchemical potentials
nuclei, ae, ap = atomic_energy_decomposition(cubes, '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/output')

In [ ]:
print(nuclei, '\n', ae, '\n', ap)

In [ ]:
from explore_qml_data import get_property, get_free_atom_data
from alchemy_tools import get_free_atom_energies
# total energy from B3LYP
total_energy = get_property('/home/misa/datasets/qm9/dsgdb9nsd_002626.xyz', 'U0')
# energies of the free atoms in qm9
free_atoms = get_free_atom_data()
# free atom energy for every atom in compound
free_en = get_free_atom_energies(nuclei[:,0], free_atoms)

In [ ]:
print(total_energy, '\n', free_atoms, '\n', free_en)

In [ ]:
from alchemy_tools import calculate_atomisation_energies 
# calculate atomisation energies
at=calculate_atomisation_energies(ae, total_energy, free_en)
at

In [ ]:
print('Atomisation energy decomposition = %f' %calculate_atomisation_energies(ae, total_energy, free_en).sum())
print('Atomisation energy qm9 = %f ' % (total_energy - (free_atoms['O']+6*free_atoms['C']+8*free_atoms['H'])) )

In [ ]:
from ase.units import Bohr
nuclei[:,[1,2,3]]*Bohr

In [ ]:
import numpy as np
np.array([nuclei[:,0],ae,at]).T